In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import issparse

# Load data
df = pd.read_csv('text_preprocessing/final_dataset.csv')
df

,Sentiment,tweet_tokens_stemmed
0,positive,"['undang', 'shanijkt', 'hitamputih', 'pemenang..."
1,positive,"['selamat', 'berbuka', 'puasa', 'semoga', 'ama..."
2,positive,"['trans', 'hitam', 'putih', 'penghargaan', 'no..."
3,positive,"['selamat', 'hitamputih']"
4,positive,"['asiknya', 'nonton', 'hitam', 'putih', 'trans']"
...,...,...
395,negative,"['banget', 'kesel', 'debat', 'pake', 'emosi', ..."
396,negative,"['miskin', 'miskin', 'sekolah', 'pungutan', 'l..."
397,negative,"['emosi', 'cepat', 'tua', 'nonton', 'emosi', '..."
398,negative,"['penampilan', 'kyk', 'preman', 'taunya', 'bki..."


In [9]:
# Convert sentiment labels to binary
def convert_label_sentimen(label_sentimen):
    return 1 if label_sentimen == "positive" else 0

df['label_sentimen'] = df['Sentiment'].apply(convert_label_sentimen)
df.drop(df.columns[[0]], axis=1, inplace=True)
df


,tweet_tokens_stemmed,label_sentimen
0,"['undang', 'shanijkt', 'hitamputih', 'pemenang...",1
1,"['selamat', 'berbuka', 'puasa', 'semoga', 'ama...",1
2,"['trans', 'hitam', 'putih', 'penghargaan', 'no...",1
3,"['selamat', 'hitamputih']",1
4,"['asiknya', 'nonton', 'hitam', 'putih', 'trans']",1
...,...,...
395,"['banget', 'kesel', 'debat', 'pake', 'emosi', ...",0
396,"['miskin', 'miskin', 'sekolah', 'pungutan', 'l...",0
397,"['emosi', 'cepat', 'tua', 'nonton', 'emosi', '...",0
398,"['penampilan', 'kyk', 'preman', 'taunya', 'bki...",0


In [10]:
X= df['tweet_tokens_stemmed'] 
y= df['label_sentimen']
#k-fold cross validation(spliting data)
kfold = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)


In [11]:
class NaiveBayes:
    def __init__(self, model_class):
        self.model_class = model_class
        self.clf = None
    
    def fit(self, X_train, y_train, selected_features):
        self.clf = self.model_class()
        if issparse(X_train):
            X_train = X_train.toarray()
        self.clf.fit(X_train[:, selected_features], y_train)
    
    def predict(self, X_test, selected_features):
        if self.clf is None:
            raise ValueError("Classifier not fitted. Please call 'fit' method first.")
        if issparse(X_test):
            X_test = X_test.toarray()
        return self.clf.predict(X_test[:, selected_features])

In [12]:
def evaluate_features(X_train, y_train, selected_features, classifier):
    clf = classifier
    cv_results = cross_val_score(clf.clf, X_train[:, selected_features], y_train, cv=kfold, scoring='accuracy')
    return cv_results.mean()

def pso_feature_selection(X_train, y_train, n_particles, inertia, global_weight, local_weight, model_class, tol=1e-5, patience=10):
    num_samples, num_features = X_train.shape
    bounds = [0, 1]
    
    num_particles = n_particles
    dimensions = num_features
    particles = np.random.rand(num_particles, dimensions)
    velocities = np.random.rand(num_particles, dimensions) * 0.1
    best_positions = particles.copy()
    best_scores = np.zeros(num_particles)

    global_best_position = np.zeros(dimensions)
    global_best_score = 0

    no_improvement_count = 0
    previous_global_best_score = 0

    while no_improvement_count < patience:
        for particle in range(num_particles):
            r1 = np.random.rand(dimensions)
            r2 = np.random.rand(dimensions)
            velocities[particle] = (inertia * velocities[particle] +
                                    global_weight * r1 * (best_positions[particle] - particles[particle]) +
                                    local_weight * r2 * (global_best_position - particles[particle]))

            particles[particle] += velocities[particle]
            particles[particle] = np.clip(particles[particle], bounds[0], bounds[1])

            selected_features = particles[particle] > 0.5
            nb = NaiveBayes(model_class)
            nb.fit(X_train, y_train, selected_features)
            accuracy = evaluate_features(X_train, y_train, selected_features, nb)

            if accuracy > best_scores[particle]:
                best_scores[particle] = accuracy
                best_positions[particle] = particles[particle].copy()

            if accuracy > global_best_score:
                global_best_score = accuracy
                global_best_position = particles[particle].copy()

        if abs(global_best_score - previous_global_best_score) < tol:
            no_improvement_count += 1
        else:
            no_improvement_count = 0
        
        previous_global_best_score = global_best_score

    return global_best_position > 0.5

In [13]:
# Define parameters for PSO optimization
n_particles = 20
inertia = 0.8
global_weight = 1
local_weight = 0.9

results = []

for model_class in [MultinomialNB, BernoulliNB, GaussianNB]:
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    for train_idx, test_idx in kfold.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        tfidf_model = TfidfVectorizer(smooth_idf=False)
        X_train_tfidf = tfidf_model.fit_transform(X_train)
        X_test_tfidf = tfidf_model.transform(X_test)

        X_train_dense = X_train_tfidf.toarray()
        selected_features = pso_feature_selection(X_train_dense, y_train, n_particles, inertia, global_weight, local_weight, model_class)

        if not np.any(selected_features):
            selected_features[0] = True

        nb = NaiveBayes(MultinomialNB)
        nb.fit(X_train_tfidf, y_train, selected_features)
        y_pred = nb.predict(X_test_tfidf.toarray(), selected_features)

        accuracy_scores.append(accuracy_score(y_test, y_pred))
        precision_scores.append(precision_score(y_test, y_pred))
        recall_scores.append(recall_score(y_test, y_pred))

    fold_results = {
        "Model": model_class.__name__,
        "Average Accuracy": np.mean(accuracy_scores),
        "Average Precision": np.mean(precision_scores),
        "Average Recall": np.mean(recall_scores)
    }

    results.append(fold_results)

results_df = pd.DataFrame(results)

print(results_df.to_string(index=False))

        Model  Average Accuracy  Average Precision  Average Recall
MultinomialNB            0.7950           0.778762           0.830
  BernoulliNB            0.7750           0.768636           0.795
   GaussianNB            0.7625           0.748291           0.805
